It consists of the repeated
application of two 3x3 convolutions (unpadded convolutions), each followed by
a rectified linear unit (ReLU) and a 2x2 max pooling operation with stride 2
for downsampling. At each downsampling step we double the number of feature
channels. Every step in the expansive path consists of an upsampling of the
feature map followed by a 2x2 convolution (“up-convolution”) that halves the
number of feature channels, a concatenation with the correspondingly cropped
feature map from the contracting path, and two 3x3 convolutions, each followed
by a ReLU. The cropping is necessary due to the loss of border pixels in
every convolution. At the final layer a 1x1 convolution is used to map each 64-
component feature vector to the desired number of classes. In total the network
has 23 convolutional layers.
To allow a seamless tiling of the output segmentation map (see Figure 2), it
is important to select the input tile size such that all 2x2 max-pooling operations
are applied to a layer with an even x- and y-size.




## Import

In [27]:
# from keras.layers import 
import numpy as np

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


# from keras import layers
# from keras.layers import Input, merge,UpSampling2D, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
# from keras.models import Model, load_model
# from keras.preprocessing import image
# from keras.utils import layer_utils
# from keras.utils.data_utils import get_file
# from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
# from keras.utils import plot_model
# from resnets_utils import *
# from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

import pandas as pd

%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [28]:
def load_dataset():
    X_train_orig = []
    Y_train_orig = []
    X_test_orig = []
    Y_test_orig = []
    classes = []
    return X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

## Model

In [31]:
def double_convolutional( X, f, stage):
    """
    Implementation of the convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window - deep
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    block = "0"
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    mp_name_base = 'mp' + str(stage) + block + '_branch'
    
    
    # First component of convolutional block
    X = Conv2D(f, (3,3), strides = (1,1), padding = 'same', name = conv_name_base + '1a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '1a')(X)
    X = Activation('relu')(X)

    # Second component of convolutional block
    X = Conv2D(f, (3,3), strides = (1,1), padding = 'same', name = conv_name_base + '1b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '1b')(X)
    X = Activation('relu')(X)

    return X

In [44]:
def unetModel(input_shape):
    """
    Implementation of the UNet.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """

    #input (572*572*1)
    X_Input = Input(input_shape)

    # Stage 1
    X1 = double_convolutional( X_Input, 64, 1)
    X1 = MaxPooling2D(pool_size=(2, 2), strides=2,name='mp1')(X1) 

    # Stage 2
    X2 = double_convolutional( X1, 128, 2)
    X2 = MaxPooling2D(pool_size=(2, 2), strides=2,name='mp2')(X2) 

    # Stage 3
    X3 = double_convolutional( X2, 256, 3)
    X3 = MaxPooling2D(pool_size=(2, 2), strides=2,name='mp3')(X3) 

    # Stage 4
    X4 = double_convolutional( X3, 512, 4)
    X4 = MaxPooling2D(pool_size=(2, 2), strides=2,name='mp4')(X4) 

    # Stage 5
    X = double_convolutional( X4, 1024, 5)

    X = UpSampling2D(size = (2,2))(X)

    X = Conv2D(512, (2,2), strides = (1,1), padding = 'same', name = 'res6', kernel_initializer = glorot_uniform(seed=0))(X)

    X = concatenate([X4,X], axis = 3)

    X = double_convolutional( X, 512, 7)

    X = UpSampling2D(size = (2,2))(X)



    X = Conv2D(256, (2,2), strides = (1,1), padding = 'same', name = 'res7', kernel_initializer = glorot_uniform(seed=0))(X)

    X = concatenate([X3,X], axis = 3)

    X = double_convolutional( X, 256, 8)

    X = UpSampling2D(size = (2,2))(X)


    X = Conv2D(128, (2,2), strides = (1,1), padding = 'same', name = 'res8', kernel_initializer = glorot_uniform(seed=0))(X)

    X = concatenate([X2,X], axis = 3)

    X = double_convolutional( X, 128, 9)

    X = UpSampling2D(size = (2,2))(X)

    X = Conv2D(64, (2,2), strides = (1,1), padding = 'same', name = 'res9', kernel_initializer = glorot_uniform(seed=0))(X)

    X = concatenate([X1,X], axis = 3)

    X = double_convolutional( X, 64, 9)

    
    X = Conv2D(1, (1,1), strides = (1,1), padding = 'same', name = 'res10', kernel_initializer = glorot_uniform(seed=0))(X)

    return X

In [ ]:
# X_train.shape[1:]
model = unetModel([580,420,3])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x=X_train,y=Y_train,epochs=40,batch_size=32)

In [ ]:
preds = model.evaluate(x=X_test,y=Y_test)

In [ ]:
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
img_path = 'images/my_image.jpg'
### END CODE HERE ###
img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(model.predict(x))

In [ ]:
happyModel.summary()

In [ ]:
plot_model(happyModel, to_file='unetModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))